<a href="https://colab.research.google.com/github/AnahitaNouri/NLP_New_First_Assignment/blob/main/New_first_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wikipedia-api

In [2]:
import wikipediaapi
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, SnowballStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk import NaiveBayesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from nltk import classify
from nltk import NaiveBayesClassifier

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
# Identify relevant annotations
geographic_keywords = ["location", "geography", "map", "country", "city"]
non_geographic_keywords = ["technology", "science", "artificial intelligence", "computer"]

In [5]:
def fetch_wikipedia_text_with_annotations(topic, keyword_list):
    # Create a Wikipedia API object
    wiki_wiki = wikipediaapi.Wikipedia('en', extract_format=wikipediaapi.ExtractFormat.WIKI, headers={'User-Agent': 'Anahita'})

    # Fetch the page for the given topic
    page = wiki_wiki.page(topic)

    if page.exists():
        # Retrieve the text content of the page
        text = page.text

        # Extract annotations or keywords from the text
        annotations = [keyword for keyword in keyword_list if keyword.lower() in text.lower()]

        return text, annotations
    else:
        print(f"Page for topic '{topic}' does not exist.")
        return None, None

# Fetch the content of Wikipedia page with annotations
topic = "Artificial intelligence"
text, annotations = fetch_wikipedia_text_with_annotations(topic, non_geographic_keywords)

if text:
    print(f"Text content for '{topic}':")
    print(text[:500])  # Print the first 500 characters of the text
    print("Annotations:", annotations)

Text content for 'Artificial intelligence':
Artificial intelligence (AI) is the intelligence of machines or software, as opposed to the intelligence of humans or other living beings. It is a field of study in computer science that develops and studies intelligent machines. Such machines may be called AIs.
AI technology is widely used throughout industry, government, and science. Some high-profile applications are: advanced web search engines (e.g., Google Search), recommendation systems (used by YouTube, Amazon, and Netflix), interacting 
Annotations: ['technology', 'science', 'artificial intelligence', 'computer']


In [6]:
def preprocess_text_with_annotations(text, annotations, stop_words=None, stemmer=None, lemmatizer=None):
    # Tokenize the text
    words = word_tokenize(text)

    # Use Snowball stop words if provided, otherwise use NLTK default stop words
    stop_words = stop_words or set(stopwords.words('english'))

    # Remove stop words
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Use Snowball stemmer if provided, otherwise use Porter Stemmer
    stemmer = stemmer or PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in filtered_words]

    # Use WordNet Lemmatizer if provided, otherwise use a basic lemmatizer
    lemmatizer = lemmatizer or WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]

    # Join the preprocessed words back into a single string
    preprocessed_text = ' '.join(lemmatized_words)

    # Incorporate annotations into the preprocessed text
    preprocessed_text += ' '.join(annotations)

    return preprocessed_text


# Usage with Bag of Words without pre-processing
bag_of_words_text = text.lower()
bag_of_words_features = {word: True for word in word_tokenize(bag_of_words_text)}

# Usage with Snowball stop word list and Porter Stemmer
snowball_stemmer = SnowballStemmer('english')
preprocessed_snowball_text = preprocess_text_with_annotations(text, annotations, stop_words=set(stopwords.words('english')), stemmer=snowball_stemmer)

# Usage with WordNet Lemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
preprocessed_lemmatized_text = preprocess_text_with_annotations(text, annotations, stop_words=set(stopwords.words('english')), lemmatizer=wordnet_lemmatizer)

# Display the results
print("\nBag of Words without pre-processing:")
print(bag_of_words_features)

print("\nPreprocessed Text with Snowball stop words and Porter Stemmer:")
print(preprocessed_snowball_text[:500])

print("\nPreprocessed Text with WordNet Lemmatizer:")
print(preprocessed_lemmatized_text[:500])


Bag of Words without pre-processing:
{'artificial': True, 'intelligence': True, '(': True, 'ai': True, ')': True, 'is': True, 'the': True, 'of': True, 'machines': True, 'or': True, 'software': True, ',': True, 'as': True, 'opposed': True, 'to': True, 'humans': True, 'other': True, 'living': True, 'beings': True, '.': True, 'it': True, 'a': True, 'field': True, 'study': True, 'in': True, 'computer': True, 'science': True, 'that': True, 'develops': True, 'and': True, 'studies': True, 'intelligent': True, 'such': True, 'may': True, 'be': True, 'called': True, 'ais': True, 'technology': True, 'widely': True, 'used': True, 'throughout': True, 'industry': True, 'government': True, 'some': True, 'high-profile': True, 'applications': True, 'are': True, ':': True, 'advanced': True, 'web': True, 'search': True, 'engines': True, 'e.g.': True, 'google': True, 'recommendation': True, 'systems': True, 'by': True, 'youtube': True, 'amazon': True, 'netflix': True, 'interacting': True, 'via': True, 'h

# Naive Bayes without Pre-processing:


In [7]:
# 'geographic' and 'non-geographic' are classes
geographic_class = "geographic"
non_geographic_class = "non-geographic"

# Training data for Naive Bayes on Bag of Words without pre-processing
training_data_bow_raw = [
    (fetch_wikipedia_text_with_annotations("Verona", geographic_keywords)[0].lower(), geographic_class),
    (fetch_wikipedia_text_with_annotations("Artificial intelligence", non_geographic_keywords)[0].lower(), non_geographic_class),
]

# Tokenizer function for raw text
def get_words_in_raw_text_bow(text):
    return word_tokenize(text)

# Feature extraction function for raw text
def extract_features_raw_bow(text):
    return {word: True for word in get_words_in_raw_text_bow(text)}

# Prepare the training set without pre-processing for Bag of Words
training_set_bow_raw = [(extract_features_raw_bow(text), label) for (text, label) in training_data_bow_raw]

# Train the Naive Bayes classifier without pre-processing for Bag of Words
naive_bayes_classifier_bow_raw = NaiveBayesClassifier.train(training_set_bow_raw)

# Usage without pre-processing for Bag of Words
test_text_bow_raw = fetch_wikipedia_text_with_annotations("Verona", geographic_keywords)[0].lower()
test_features_bow_raw = extract_features_raw_bow(test_text_bow_raw)
classification_bow_raw = naive_bayes_classifier_bow_raw.classify(test_features_bow_raw)

print(f"\nPredicted class for the test text without pre-processing for Bag of Words: {classification_bow_raw}")

test_text_bow_raw = fetch_wikipedia_text_with_annotations("Data Science", non_geographic_keywords)[0].lower()
test_features_bow_raw = extract_features_raw_bow(test_text_bow_raw)
classification_bow_raw = naive_bayes_classifier_bow_raw.classify(test_features_bow_raw)

print(f"Predicted class for the test text without pre-processing for Bag of Words: {classification_bow_raw}")





Predicted class for the test text without pre-processing for Bag of Words: geographic
Predicted class for the test text without pre-processing for Bag of Words: non-geographic


# Naive Bayes with Pre-processing:


In [8]:
# Training data for Naive Bayes with Snowball stop words and Porter Stemmer
training_data_bow_snowball = [
    (preprocess_text_with_annotations(fetch_wikipedia_text_with_annotations("Verona", geographic_keywords)[0], geographic_keywords, stop_words=set(stopwords.words('english')), stemmer=snowball_stemmer).lower(), geographic_class),
    (preprocess_text_with_annotations(fetch_wikipedia_text_with_annotations("Artificial intelligence", non_geographic_keywords)[0], non_geographic_keywords, stop_words=set(stopwords.words('english')), stemmer=snowball_stemmer).lower(), non_geographic_class),
]

# Tokenizer function for preprocessed text
def get_words_in_preprocessed_text_bow(text):
    return word_tokenize(text)

# Feature extraction function for preprocessed text
def extract_features_preprocessed_bow(text):
    return {word: True for word in get_words_in_preprocessed_text_bow(text)}

# Prepare the training set with Snowball stop words and Porter Stemmer for Bag of Words
training_set_bow_snowball = [(extract_features_preprocessed_bow(text), label) for (text, label) in training_data_bow_snowball]

# Train the Naive Bayes classifier with Snowball stop words and Porter Stemmer for Bag of Words
naive_bayes_classifier_bow_snowball = NaiveBayesClassifier.train(training_set_bow_snowball)

# Usage with Snowball stop words and Porter Stemmer for Bag of Words
test_text_bow_snowball = preprocess_text_with_annotations(fetch_wikipedia_text_with_annotations("Verona", geographic_keywords)[0], geographic_keywords, stop_words=set(stopwords.words('english')), stemmer=snowball_stemmer).lower()
test_features_bow_snowball = extract_features_preprocessed_bow(test_text_bow_snowball)
classification_bow_snowball = naive_bayes_classifier_bow_snowball.classify(test_features_bow_snowball)

print(f"\nPredicted class for the preprocessed test text with Snowball stop words and Porter Stemmer for Bag of Words: {classification_bow_snowball}")

test_text_bow_snowball = preprocess_text_with_annotations(fetch_wikipedia_text_with_annotations("Data Science", non_geographic_keywords)[0], non_geographic_keywords, stop_words=set(stopwords.words('english')), stemmer=snowball_stemmer).lower()
test_features_bow_snowball = extract_features_preprocessed_bow(test_text_bow_snowball)
classification_bow_snowball = naive_bayes_classifier_bow_snowball.classify(test_features_bow_snowball)

print(f"Predicted class for the preprocessed test text with Snowball stop words and Porter Stemmer for Bag of Words: {classification_bow_snowball}")


Predicted class for the preprocessed test text with Snowball stop words and Porter Stemmer for Bag of Words: geographic
Predicted class for the preprocessed test text with Snowball stop words and Porter Stemmer for Bag of Words: non-geographic


#  Logistic Regression

In [9]:
# Training data for Logistic Regression with pre-processing from Naive Bayes
training_data_logistic_preprocessed = [
    (preprocess_text_with_annotations(fetch_wikipedia_text_with_annotations("Verona", geographic_keywords)[0], geographic_keywords, stop_words=set(stopwords.words('english')), stemmer=snowball_stemmer).lower(), geographic_class),
    (preprocess_text_with_annotations(fetch_wikipedia_text_with_annotations("Artificial intelligence", non_geographic_keywords)[0], non_geographic_keywords, stop_words=set(stopwords.words('english')), stemmer=snowball_stemmer).lower(), non_geographic_class),
]

# Tokenizer function for preprocessed text (same as used for Naive Bayes)
def get_words_in_preprocessed_text_logistic(text):
    return word_tokenize(text)

# Feature extraction function for preprocessed text (same as used for Naive Bayes)
def extract_features_preprocessed_logistic(text):
    return ' '.join(get_words_in_preprocessed_text_logistic(text))

# Prepare the training set for Logistic Regression with pre-processing
training_set_logistic_preprocessed = [(extract_features_preprocessed_logistic(text), label) for (text, label) in training_data_logistic_preprocessed]

# Create TF-IDF vectors from the training set for Logistic Regression with pre-processing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Separate features (X) and labels (y) for Logistic Regression with pre-processing
X_train_logistic_preprocessed = [text for (text, _) in training_set_logistic_preprocessed]
y_train_logistic_preprocessed = [label for (_, label) in training_set_logistic_preprocessed]

# Create TF-IDF vectors from the training set for Logistic Regression with pre-processing
tfidf_vectorizer_logistic_preprocessed = TfidfVectorizer()
X_train_tfidf_logistic_preprocessed = tfidf_vectorizer_logistic_preprocessed.fit_transform(X_train_logistic_preprocessed)

# Train the Logistic Regression classifier with pre-processing
logistic_regression_classifier_preprocessed = LogisticRegression()
logistic_regression_classifier_preprocessed.fit(X_train_tfidf_logistic_preprocessed, y_train_logistic_preprocessed)

# Usage for Logistic Regression with pre-processing
test_text_logistic_preprocessed = preprocess_text_with_annotations(fetch_wikipedia_text_with_annotations("Verona", geographic_keywords)[0], geographic_keywords, stop_words=set(stopwords.words('english')), stemmer=snowball_stemmer).lower()
test_features_logistic_preprocessed = tfidf_vectorizer_logistic_preprocessed.transform([extract_features_preprocessed_logistic(test_text_logistic_preprocessed)])
classification_logistic_preprocessed = logistic_regression_classifier_preprocessed.predict(test_features_logistic_preprocessed)

print(f"\nPredicted class for the test text using Logistic Regression with pre-processing: {classification_logistic_preprocessed[0]}")

test_text_logistic_preprocessed = preprocess_text_with_annotations(fetch_wikipedia_text_with_annotations("Data Science", non_geographic_keywords)[0], non_geographic_keywords, stop_words=set(stopwords.words('english')), stemmer=snowball_stemmer).lower()
test_features_logistic_preprocessed = tfidf_vectorizer_logistic_preprocessed.transform([extract_features_preprocessed_logistic(test_text_logistic_preprocessed)])
classification_logistic_preprocessed = logistic_regression_classifier_preprocessed.predict(test_features_logistic_preprocessed)

print(f"Predicted class for the test text using Logistic Regression with pre-processing: {classification_logistic_preprocessed[0]}")

# Training data for Logistic Regression without pre-processing
training_data_logistic_raw = [
    (fetch_wikipedia_text_with_annotations("Verona", geographic_keywords)[0].lower(), geographic_class),
    (fetch_wikipedia_text_with_annotations("Artificial intelligence", non_geographic_keywords)[0].lower(), non_geographic_class),
]

# Tokenizer function for raw text (same as used for Naive Bayes)
def get_words_in_raw_text_logistic(text):
    return word_tokenize(text)

# Feature extraction function for raw text (same as used for Naive Bayes)
def extract_features_raw_logistic(text):
    return ' '.join(get_words_in_raw_text_logistic(text))

# Prepare the training set for Logistic Regression without pre-processing
training_set_logistic_raw = [(extract_features_raw_logistic(text), label) for (text, label) in training_data_logistic_raw]

# Create TF-IDF vectors from the training set for Logistic Regression without pre-processing
X_train_logistic_raw = [text for (text, _) in training_set_logistic_raw]
y_train_logistic_raw = [label for (_, label) in training_set_logistic_raw]

tfidf_vectorizer_logistic_raw = TfidfVectorizer()
X_train_tfidf_logistic_raw = tfidf_vectorizer_logistic_raw.fit_transform(X_train_logistic_raw)

# Train the Logistic Regression classifier without pre-processing
logistic_regression_classifier_raw = LogisticRegression()
logistic_regression_classifier_raw.fit(X_train_tfidf_logistic_raw, y_train_logistic_raw)

# Usage for Logistic Regression without pre-processing
test_text_logistic_raw = fetch_wikipedia_text_with_annotations("Verona", geographic_keywords)[0].lower()
test_features_logistic_raw = tfidf_vectorizer_logistic_raw.transform([extract_features_raw_logistic(test_text_logistic_raw)])
classification_logistic_raw = logistic_regression_classifier_raw.predict(test_features_logistic_raw)

print(f"\nPredicted class for the test text using Logistic Regression without pre-processing: {classification_logistic_raw[0]}")

test_text_logistic_raw = fetch_wikipedia_text_with_annotations("Data Science", non_geographic_keywords)[0].lower()
test_features_logistic_raw = tfidf_vectorizer_logistic_raw.transform([extract_features_raw_logistic(test_text_logistic_raw)])
classification_logistic_raw = logistic_regression_classifier_raw.predict(test_features_logistic_raw)

print(f"Predicted class for the test text using Logistic Regression without pre-processing: {classification_logistic_raw[0]}")



Predicted class for the test text using Logistic Regression with pre-processing: geographic
Predicted class for the test text using Logistic Regression with pre-processing: non-geographic

Predicted class for the test text using Logistic Regression without pre-processing: geographic
Predicted class for the test text using Logistic Regression without pre-processing: non-geographic
